<a href="https://colab.research.google.com/github/supriya5706/JumpPlus/blob/main/Data_lake_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import sparksession to work with dataframe
from pyspark.sql import SparkSession
#create a SparkSession object in PySpark 
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark 

In [ ]:
#importing pyspark function to perform all the operations on DataFrame
from pyspark.sql.functions import *


In [ ]:
#importing data set
books = spark.read.csv("/content/data_lake.cvs", header=True, inferSchema=True) 
books

DataFrame[_c0: int, Unnamed: 0: double, item_id: double, name: string, category: string, price5: double, old_price: string, sellable_online: boolean, link: string, other_colors: string, short_description: string, designer: string, depth: string, height: double, width: double, Pokemon: string, Card Type: string, Generation: string, Card Number: string, Price19: double, ranks: double, title: string, no_of_reviews: string, ratings: string, author: string, cover_type: string, year: string, genre: string, date: string, product: string, market: string, size: string]

In [ ]:
#mounting google drive to Colab environment 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
type(books)


pyspark.sql.dataframe.DataFrame

In [ ]:
#displaying data frame 
books.show()

+---+----------+-----------+--------------------+-------------+------+------------+---------------+--------------------+------------+--------------------+--------------------+-----+------+-----+-------+---------+----------+-----------+-------+-----+-----+-------------+-------+------+----------+----+-----+----+-------+------+----+
|_c0|Unnamed: 0|    item_id|                name|     category|price5|   old_price|sellable_online|                link|other_colors|   short_description|            designer|depth|height|width|Pokemon|Card Type|Generation|Card Number|Price19|ranks|title|no_of_reviews|ratings|author|cover_type|year|genre|date|product|market|size|
+---+----------+-----------+--------------------+-------------+------+------------+---------------+--------------------+------------+--------------------+--------------------+-----+------+-----+-------+---------+----------+-----------+-------+-----+-----+-------------+-------+------+----------+----+-----+----+-------+------+----+
|  0

In [ ]:
 # Select the columns for books dataset
book_df = books.select(["title", "author","ratings","genre","price5","cover_type","ranks","year","no_of_reviews"])
book_df.show()

+-----+------+-------+-----+------+----------+-----+----+-------------+
|title|author|ratings|genre|price5|cover_type|ranks|year|no_of_reviews|
+-----+------+-------+-----+------+----------+-----+----+-------------+
| null|  null|   null| null| 265.0|      null| null|null|         null|
| null|  null|   null| null| 995.0|      null| null|null|         null|
| null|  null|   null| null|2095.0|      null| null|null|         null|
| null|  null|   null| null|  69.0|      null| null|null|         null|
| null|  null|   null| null| 225.0|      null| null|null|         null|
| null|  null|   null| null| 345.0|      null| null|null|         null|
| null|  null|   null| null| 129.0|      null| null|null|         null|
| null|  null|   null| null| 195.0|      null| null|null|         null|
| null|  null|   null| null| 129.0|      null| null|null|         null|
| null|  null|   null| null|2176.0|      null| null|null|         null|
| null|  null|   null| null| 149.0|      null| null|null|       

  ---------------------------Data Cleaning---------------------------------

In [ ]:
# Find Count of Null, None, NaN in df columns
from pyspark.sql.functions import col,isnan, when, count
book_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in book_df.columns]
   ).show()

+-----+------+-------+-----+------+----------+-----+-----+-------------+
|title|author|ratings|genre|price5|cover_type|ranks| year|no_of_reviews|
+-----+------+-------+-----+------+----------+-----+-----+-------------+
|91229| 91229|  91228|91227| 40829|     91229|91228|91224|        91228|
+-----+------+-------+-----+------+----------+-----+-----+-------------+



In [ ]:
#Removing null values from DataFrame
book_df = book_df.dropna(subset=["title", "author","ratings","genre","price5","cover_type","ranks","year","no_of_reviews"])


In [ ]:
#show data set afetr dropping null values
book_df.show()

+--------------------+----------------+-------+-----------+------+----------+-----+------+-------------+
|               title|          author|ratings|      genre|price5|cover_type|ranks|  year|no_of_reviews|
+--------------------+----------------+-------+-----------+------+----------+-----+------+-------------+
|     The Lost Symbol|       Dan Brown|    4.4|    Fiction| 12.49| Hardcover|  1.0|2009.0|       16,118|
|The Shack: Where ...|William P. Young|    4.7|    Fiction|  13.4| Paperback|  2.0|2009.0|       23,392|
|Liberty and Tyran...|   Mark R. Levin|    4.8|Non Fiction|  9.93| Hardcover|  3.0|2009.0|        5,036|
|Breaking Dawn (Th...| Stephenie Meyer|    4.7|    Fiction|  14.3| Hardcover|  4.0|2009.0|       16,912|
|Going Rogue: An A...|     Sarah Palin|    4.6|Non Fiction|  9.99| Hardcover|  5.0|2009.0|        1,572|
| StrengthsFinder 2.0|          Gallup|    4.1|Non Fiction| 18.29| Hardcover|  6.0|2009.0|        7,082|
|            The Help|Kathryn Stockett|    4.8|    Fict

In [ ]:
# Count the number of rows in the original DataFrame
num_rows_original = book_df.count()
num_rows_original

1285

In [ ]:
# Find duplicatesbased on the title column
df_duplicates = book_df.dropDuplicates(["Book_Title"])


In [ ]:
# Count the number of rows in the new DataFrame
num_rows_duplicates = df_duplicates.count()
num_rows_duplicates

743

In [ ]:
#changing data set column name 

# create a dictionary with the old column names as keys and new column names as values
replace_name_dict = {'title': 'Book_Title', 'author': 'Book_Author','ratings':'Book_Rating','genre':'Book_Genre','price5':'Book_Price','cover_type':'Book_Cover_Type','ranks':'Book_Ranks','year':'Book_Release_Year','no_of_reviews':'Book_Review_Number'}

# use a for loop to iterate over the dictionary and rename the columns using withColumnRenamed()
for old_name, new_name in replace_name_dict.items():
   book_df = book_df.withColumnRenamed(old_name, new_name)


In [ ]:
book_df.show()

+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|          Book_Title|     Book_Author|Book_Rating| Book_Genre|Book_Price|Book_Cover_Type|Book_Ranks|Book_Release_Year|Book_Review_Number|
+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|     The Lost Symbol|       Dan Brown|        4.4|    Fiction|     12.49|      Hardcover|       1.0|           2009.0|            16,118|
|The Shack: Where ...|William P. Young|        4.7|    Fiction|      13.4|      Paperback|       2.0|           2009.0|            23,392|
|Liberty and Tyran...|   Mark R. Levin|        4.8|Non Fiction|      9.93|      Hardcover|       3.0|           2009.0|             5,036|
|Breaking Dawn (Th...| Stephenie Meyer|        4.7|    Fiction|      14.3|      Hardcover|       4.0|           2009.0|            16,912|
|Going Rogue: An A...|     

In [ ]:
#removing unwated obervation
# Filter out bad data 
book_df = book_df.filter(col("Book_Author") != "22,268")
book_df = book_df.filter(col("Book_Author") != "4.4")


In [ ]:
#displayed DataFrame after deleting raws
book_df.show()

+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|          Book_Title|     Book_Author|Book_Rating| Book_Genre|Book_Price|Book_Cover_Type|Book_Ranks|Book_Release_Year|Book_Review_Number|
+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|     The Lost Symbol|       Dan Brown|        4.4|    Fiction|     12.49|      Hardcover|       1.0|           2009.0|            16,118|
|The Shack: Where ...|William P. Young|        4.7|    Fiction|      13.4|      Paperback|       2.0|           2009.0|            23,392|
|Liberty and Tyran...|   Mark R. Levin|        4.8|Non Fiction|      9.93|      Hardcover|       3.0|           2009.0|             5,036|
|Breaking Dawn (Th...| Stephenie Meyer|        4.7|    Fiction|      14.3|      Hardcover|       4.0|           2009.0|            16,912|
|Going Rogue: An A...|     

In [ ]:
#checking columns types
book_df.describe()

DataFrame[summary: string, Book_Title: string, Book_Author: string, Book_Rating: string, Book_Genre: string, Book_Price: string, Book_Cover_Type: string, Book_Ranks: string, Book_Release_Year: string, Book_Review_Number: string]

In [ ]:
#changing the year column from float to int 
book_df = book_df.withColumn("Book_Release_year",col("Book_Release_year").cast("int"))
book_df.show()


+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|          Book_Title|     Book_Author|Book_Rating| Book_Genre|Book_Price|Book_Cover_Type|Book_Ranks|Book_Release_year|Book_Review_Number|
+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|     The Lost Symbol|       Dan Brown|        4.4|    Fiction|     12.49|      Hardcover|       1.0|             2009|            16,118|
|The Shack: Where ...|William P. Young|        4.7|    Fiction|      13.4|      Paperback|       2.0|             2009|            23,392|
|Liberty and Tyran...|   Mark R. Levin|        4.8|Non Fiction|      9.93|      Hardcover|       3.0|             2009|             5,036|
|Breaking Dawn (Th...| Stephenie Meyer|        4.7|    Fiction|      14.3|      Hardcover|       4.0|             2009|            16,912|
|Going Rogue: An A...|     

In [ ]:
#changing rank column into integer
book_df = book_df.withColumn("Book_Ranks", col("Book_Ranks").cast("int"))
book_df.show()

+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|          Book_Title|     Book_Author|Book_Rating| Book_Genre|Book_Price|Book_Cover_Type|Book_Ranks|Book_Release_year|Book_Review_Number|
+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|     The Lost Symbol|       Dan Brown|        4.4|    Fiction|     12.49|      Hardcover|         1|             2009|            16,118|
|The Shack: Where ...|William P. Young|        4.7|    Fiction|      13.4|      Paperback|         2|             2009|            23,392|
|Liberty and Tyran...|   Mark R. Levin|        4.8|Non Fiction|      9.93|      Hardcover|         3|             2009|             5,036|
|Breaking Dawn (Th...| Stephenie Meyer|        4.7|    Fiction|      14.3|      Hardcover|         4|             2009|            16,912|
|Going Rogue: An A...|     

In [ ]:
# Show the dataframe after deleting a rows
book_df.show()

+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|          Book_Title|     Book_Author|Book_Rating| Book_Genre|Book_Price|Book_Cover_Type|Book_Ranks|Book_Release_year|Book_Review_Number|
+--------------------+----------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|     The Lost Symbol|       Dan Brown|        4.4|    Fiction|     12.49|      Hardcover|         1|             2009|            16,118|
|The Shack: Where ...|William P. Young|        4.7|    Fiction|      13.4|      Paperback|         2|             2009|            23,392|
|Liberty and Tyran...|   Mark R. Levin|        4.8|Non Fiction|      9.93|      Hardcover|         3|             2009|             5,036|
|Breaking Dawn (Th...| Stephenie Meyer|        4.7|    Fiction|      14.3|      Hardcover|         4|             2009|            16,912|
|Going Rogue: An A...|     

-------------------Data Analysing----------------------

In [ ]:
#print schema
book_df.printSchema()

root
 |-- Book_Title: string (nullable = true)
 |-- Book_Author: string (nullable = true)
 |-- Book_Rating: string (nullable = true)
 |-- Book_Genre: string (nullable = true)
 |-- Book_Price: double (nullable = true)
 |-- Book_Cover_Type: string (nullable = true)
 |-- Book_Ranks: integer (nullable = true)
 |-- Book_Release_year: integer (nullable = true)
 |-- Book_Review_Number: string (nullable = true)



In [ ]:
#Rating of book by book title
book_df.groupby("Book_Title","Book_Rating").count().show()

+--------------------+-----------+-----+
|          Book_Title|Book_Rating|count|
+--------------------+-----------+-----+
|Magnus Chase and ...|        4.7|    1|
|Burn After Writin...|        4.6|    1|
|The Serpent's Sha...|        4.8|    1|
|Miss Peregrine's ...|        4.5|    1|
|The Omnivore's Di...|        4.6|    2|
|The 4 Hour Body: ...|        4.4|    2|
|Go Set a Watchman...|        3.7|    1|
|Magnus Chase and ...|        4.7|    1|
|Fire and Fury: In...|        4.2|    1|
|A Dance with Drag...|        4.5|    2|
|What Happened to ...|        4.7|    1|
|Keep Sharp: Build...|        4.7|    1|
|Liberty and Tyran...|        4.8|    1|
|Harry Potter and ...|        4.9|    2|
|Change Sings: A C...|        4.9|    1|
|Towers of Midnigh...|        4.8|    1|
|Last Week Tonight...|        4.9|    1|
|   Little Blue Truck|        4.9|    2|
|The Five Dysfunct...|        4.6|    8|
|Diary of a Wimpy ...|        4.8|    1|
+--------------------+-----------+-----+
only showing top

In [ ]:
#showing the books with the highest rating
highest_rating = book_df.orderBy("Book_Rating", ascending=False).show()

+--------------------+--------------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|          Book_Title|         Book_Author|Book_Rating| Book_Genre|Book_Price|Book_Cover_Type|Book_Ranks|Book_Release_year|Book_Review_Number|
+--------------------+--------------------+-----------+-----------+----------+---------------+----------+-----------------+------------------+
|Jesus Calling, Pa...|         Sarah Young|        4.9|Non Fiction|      7.53|      Hardcover|        21|             2011|            31,196|
|The Very Hungry C...|          Eric Carle|        4.9|    Fiction|      4.93|     Board book|        39|             2014|            54,008|
|   Every Thing On It|    Shel Silverstein|        4.9|    Fiction|     11.99|      Hardcover|        56|             2011|             1,753|
|Jesus Calling, Pa...|         Sarah Young|        4.9|Non Fiction|      7.53|      Hardcover|        72|             2010|            31,196|

In [ ]:
#count of books for each author 
author_count = book_df.groupBy("Book_Author").count().orderBy("count", ascending=False).show()


+--------------------+-----+
|         Book_Author|count|
+--------------------+-----+
|        Rick Riordan|   20|
|     Suzanne Collins|   17|
|         Jeff Kinney|   17|
|        John Grisham|   15|
|        Gary Chapman|   13|
|           Dr. Seuss|   13|
|              Gallup|   13|
|American Psycholo...|   13|
|     Bill Martin Jr.|   12|
|    Malcolm Gladwell|   12|
|        Paulo Coelho|   11|
|   The College Board|   11|
|          Dav Pilkey|   11|
|         Brené Brown|   11|
|    Stephen R. Covey|   10|
|         Rob Elliott|   10|
|     Stephenie Meyer|   10|
|     Don Miguel Ruiz|   10|
|          Harper Lee|    9|
|          John Green|    9|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#displaying avg price by book's title
avg_price_by_book = book_df.groupBy("Book_Title").agg({"Book_Price": "avg"}).orderBy("avg(Book_Price)", ascending=False).show(15)


+--------------------+---------------+
|          Book_Title|avg(Book_Price)|
+--------------------+---------------+
|Diagnostic and St...|          144.0|
|   Sookie Stackhouse|          82.15|
|MyMathLab: Studen...|          75.49|
|Minecraft: The Co...|           65.0|
|Divergent / Insur...|           62.5|
|The Hunger Games ...|           59.9|
|The Twilight Saga...|           58.9|
|Game of Thrones B...|          54.98|
|The World of Ice ...|          53.99|
|             Decoded|          44.99|
|A Patriot's Histo...|           44.5|
|The Official Guid...|          39.99|
|Harry Potter Pape...|          38.86|
|Becoming a Supple...|          35.47|
|The Belly Fat Cur...|           35.3|
+--------------------+---------------+
only showing top 15 rows



In [ ]:
#average rating by book gener
avg_rating_by_gene = book_df.groupBy('Book_Genre').agg(avg('Book_Rating'))
avg_rating_by_gene.show()

+-----------+-----------------+
| Book_Genre| avg(Book_Rating)|
+-----------+-----------------+
|    Fiction|4.673750000000019|
|Non Fiction|4.633841886269072|
+-----------+-----------------+

